<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL2.0/blob/main/08_upgrade_date/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#08. Upgrade Rate by Product Action
**Returning the fraction of users, rounded to two
decimal places, who first accessed feature
two (type: F2 in events table) and
upgraded to premium within the first 30
days of signing up.**

In [1]:
!pip install polars
import pandas as pd
import numpy as  np
import polars as pl

In [2]:
data1 = {'user_id'   : [1,2,3,4,5,6,7],
         'name'      : ['John','Jane','Jill','Josh','Jean','Justin','Jeremy'],
         'join_date' : ['14-feb-20',
                        '14-feb-20',
                        '15-feb-20',
                        '15-feb-20',
                        '16-feb-20',
                        '17-feb-20',
                        '18-feb-20']
        }

data2 = {'user_id'     : [1,2,2,3,4,1,3],
         'Type'        : ['F1','F2','P','F2','F2','P','P'],
         'access_date' : ['1-mar-20',
                          '2-mar-20',
                          '12-mar-20',
                          '15-mar-20',
                          '15-mar-20',
                          '16-mar-20',
                          '22-mar-20']
        }

pd_users=pd.DataFrame(data1)
pd_events=pd.DataFrame(data2)
lf_users=pl.LazyFrame(data1)
lf_events=pl.LazyFrame(data2)

In [3]:
pd_users['join_date']=(pd.to_datetime(pd_users['join_date']
                                      ,format="%d-%b-%y"
                          )
)
pd_events['access_date']=(pd.to_datetime(pd_events['access_date']
                                         ,format="%d-%b-%y"
                             )
)

In [4]:
print(f'----PANDAS----\nusers table:\n{pd_users}')

----PANDAS----
users table:
   user_id    name  join_date
0        1    John 2020-02-14
1        2    Jane 2020-02-14
2        3    Jill 2020-02-15
3        4    Josh 2020-02-15
4        5    Jean 2020-02-16
5        6  Justin 2020-02-17
6        7  Jeremy 2020-02-18


In [5]:
print(f'events table:\n{pd_events}')

events table:
   user_id Type access_date
0        1   F1  2020-03-01
1        2   F2  2020-03-02
2        2    P  2020-03-12
3        3   F2  2020-03-15
4        4   F2  2020-03-15
5        1    P  2020-03-16
6        3    P  2020-03-22


In [6]:
pd_df1=(pd.merge(pd_users.drop(columns='name'),
                 pd_events.query("Type=='F2'")
                          .drop(columns=['Type',
                                         'access_date'
                                ]
                           ),
                 on='user_id',
                 how='inner'
           )
)
print(f'Identifying F2 users:\n{pd_df1}')

Identifying F2 users:
   user_id  join_date
0        2 2020-02-14
1        3 2020-02-15
2        4 2020-02-15


In [7]:
pd_df2=(pd_events.query("Type=='P'")
                 .drop(columns='Type')
)
print(f'Identifying access as premium users:\n{pd_df2}')

Identifying access as premium users:
   user_id access_date
2        2  2020-03-12
5        1  2020-03-16
6        3  2020-03-22


In [8]:
pd_df=(pd.merge(pd_users.drop(columns='name'),
                pd_events.query("Type=='F2'")
                         .drop(columns=['Type',
                                        'access_date'
                               ]
                          ),
                on='user_id',
                how='inner'
          )
         .merge(pd_events.query("Type=='P'")
                         .drop(columns='Type'),
                on='user_id',
                how='left'
          )
)
pd_df['duration']=(pd_df['access_date']-pd_df['join_date']).dt.days
query=pd_df[['user_id','duration']]
print(f'Elapsed time between\njoin date and access as premium:\n{query}')
fraction1=round((pd_df['duration']<=30.0).mean(),2)
print(f'\nfraction of users that updated\nwithin the first 30 days:\n{fraction1}')

Elapsed time between
join date and access as premium:
   user_id  duration
0        2      27.0
1        3      36.0
2        4       NaN

fraction of users that updated
within the first 30 days:
0.33


In [9]:
lf_users=(lf_users.with_columns(pl.col('join_date')
                                  .str
                                  .strptime(pl.Date,
                                            format="%d-%b-%y"
                                   )
                   )
)
lf_events=(lf_events.with_columns(pl.col('access_date')
                                    .str
                                    .strptime(pl.Date,
                                              format="%d-%b-%y"
                                     )
                     )
)
sample_users=lf_users.head(5)
sample_events=lf_events.head(5)

In [10]:
query1=sample_users.collect()
print(f'----POLARS----\nusers table (sample):\n{query1}')

----POLARS----
users table (sample):
shape: (5, 3)
┌─────────┬──────┬────────────┐
│ user_id ┆ name ┆ join_date  │
│ ---     ┆ ---  ┆ ---        │
│ i64     ┆ str  ┆ date       │
╞═════════╪══════╪════════════╡
│ 1       ┆ John ┆ 2020-02-14 │
│ 2       ┆ Jane ┆ 2020-02-14 │
│ 3       ┆ Jill ┆ 2020-02-15 │
│ 4       ┆ Josh ┆ 2020-02-15 │
│ 5       ┆ Jean ┆ 2020-02-16 │
└─────────┴──────┴────────────┘


In [11]:
query2=sample_events.collect()
print(f'----POLARS----\nevents table (sample):\n{query2}')

----POLARS----
events table (sample):
shape: (5, 3)
┌─────────┬──────┬─────────────┐
│ user_id ┆ Type ┆ access_date │
│ ---     ┆ ---  ┆ ---         │
│ i64     ┆ str  ┆ date        │
╞═════════╪══════╪═════════════╡
│ 1       ┆ F1   ┆ 2020-03-01  │
│ 2       ┆ F2   ┆ 2020-03-02  │
│ 2       ┆ P    ┆ 2020-03-12  │
│ 3       ┆ F2   ┆ 2020-03-15  │
│ 4       ┆ F2   ┆ 2020-03-15  │
└─────────┴──────┴─────────────┘


In [12]:
pl_lf1=(sample_users.select(pl.col('user_id'),
                            pl.col('join_date')
                    )
                   .join(sample_events.filter(pl.col('Type')=='F2')
                                      .select(pl.col('user_id')),
                         on='user_id',
                         how='inner'
                    )
).collect()
print(f'Identifying F2 users (SAMPLE):\n{pl_lf1}')

Identifying F2 users (SAMPLE):
shape: (3, 2)
┌─────────┬────────────┐
│ user_id ┆ join_date  │
│ ---     ┆ ---        │
│ i64     ┆ date       │
╞═════════╪════════════╡
│ 2       ┆ 2020-02-14 │
│ 3       ┆ 2020-02-15 │
│ 4       ┆ 2020-02-15 │
└─────────┴────────────┘


In [13]:
pl_lf2=(sample_events.filter(pl.col('Type')=='P')
                     .select(pl.col('user_id'),
                             pl.col('access_date')
                      )
).collect()
print(f'Identifying access as premium (SAMPLE):\n{pl_lf2}')

Identifying access as premium (SAMPLE):
shape: (1, 2)
┌─────────┬─────────────┐
│ user_id ┆ access_date │
│ ---     ┆ ---         │
│ i64     ┆ date        │
╞═════════╪═════════════╡
│ 2       ┆ 2020-03-12  │
└─────────┴─────────────┘


In [14]:
pl_lf=(lf_users.select(pl.col('user_id'),
                       pl.col('join_date')
                )
               .join(lf_events.filter(pl.col('Type')=='F2')
                              .select(pl.col('user_id')),
                     on='user_id',
                     how='inner'
                )
               .join(lf_events.filter(pl.col('Type')=='P')
                              .select(pl.col('user_id'),
                                      pl.col('access_date')
                               ),
                     on='user_id',
                     how='left'
                )
               .with_columns(duration=(pl.col('access_date')-pl.col('join_date'))
                                      .dt.total_days()
                )
               .select(pl.col('user_id'),
                       pl.col('duration')
                )
).collect()
print(f'Elapsed time between\njoin date and access as premium:\n{pl_lf}')
fraction2=(pl_lf.select((pl.col('duration')<=30.0).fill_null(False)
                                                  .mean()
                                                  .round(2)
                 )
)
print(f'\nfraction of users that updated\nwithin the first 30 days:\n{fraction2}')

Elapsed time between
join date and access as premium:
shape: (3, 2)
┌─────────┬──────────┐
│ user_id ┆ duration │
│ ---     ┆ ---      │
│ i64     ┆ i64      │
╞═════════╪══════════╡
│ 2       ┆ 27       │
│ 3       ┆ 36       │
│ 4       ┆ null     │
└─────────┴──────────┘

fraction of users that updated
within the first 30 days:
shape: (1, 1)
┌──────────┐
│ duration │
│ ---      │
│ f64      │
╞══════════╡
│ 0.33     │
└──────────┘
